# Attention的介绍

<img src="images/nums_epoches.png" />


当我们去看张照片的时候，我们首先就是先去看整体，这里有车、有街道、还有很多的广告牌，不知道大家是否有感受到，当我开始描述这些的时候，其实就是我把注意力放在了这些上面了。那当我们想要跟深入了解这张图片的时候，我就要把注意力放的更聚焦。比如说，我想知道这是拍的哪里，那我可能会试着去看看广告牌上的文字，看这些文字是不是能给我一些启示。

我们可能会试着把注意力放到不同的区域，那我们就能够得到更多的关于不同角度的信息。这些信息，正是我们希望在图像处理的时候希望得到的。



比如说上面的这句话，“她正在吃一个绿色的苹果”，这里我们可以比较清楚的看到，“吃”和“苹果”有很强的联系，那我们就希望在处理吃这个单词的时候，能够在语义中，包含一定的苹果的信息，这样能够帮助我们更好的理解“吃”这个动作。“绿色的”和“苹果”也是一样的，attention的机制能够帮助我们在处理单个的token的时候，带有一定的上下文信息。这就像是一种“软性记忆”一样，帮助我们记住上下文中包含的信息。

当我们看一篇文章的时候，其实也是类似的。我们从拿到一篇文章开始，首先关注的也只是一些关键性的词语，这些关键性的词语，就能够帮助我们快速的判断文章的内容和结构。这些场景就是我们在一些具体场景中对attention的应用。

假设我们的时间序列： $X = \{x_0, ..., x_n\}$, 我们把它放到坐标轴上，这些点是我们从整体数据中采样出来的，这里有很多的噪音（noisy），我们想办法能不能通过一些方法，得到这些数据的更好的表示，从而能够使噪音减少。

**Re-Weighting**: 让我们的这些点都包含一些其他点的信息，使得所有的数据能够更平滑一些. 我们定义这些**re-weighting**的参数为 $w_i = {w_{0i}, ..., w_{ni}}$, 我们使用这些weights就能够得到一个点的新的表示$y_i = w_i * X$.

$$Y = \{ w_i * x_i | w_i \in W x_i \in X\}$$

![](images/re_weight.png)

这里的$w_i$， 其实本质上是对所有输入 X的一个总结，然后得到的一个对应的值。 


Self-attention 是一个序列到序列的操作：一个向量的序列作为输入，一个向量的序列作为输出。 我们把输入的序列定义为 $x_1, x_2, ..., x_n$, 并且与它相关的输出向量是 $y_1, y_2, ..., y_n$. 这两个向量的维度都是$k$. 那么对于每一个点$x_i$, 我们都可以通过一个不同的权重值，来将它转化为一个新的序列，这个新的序列就可能是我们原始序列的一个更好的表示，这些 $w_i$ 就是一组attention的值.  它能够帮我们找到子序列和全局的attention的关系，也就是找到权重值 $w_i$.

# Self-attention

Self-attention就本质上是一种特殊的attention。attention机制能够帮我们找到子序列和全局的attention的关系，也就是找到权重值 $w_i$. self-attention对于attention的变化，其实就是寻找权重值$w_i$的过程不同。

为了能够产生输出的向量$y_i$，self-attention其实是对所有的输入做了一个加权平均的操作: $y_i = \sum^{n}_{j} w_{i,j} * x_j$. $n$代表整个序列的长度, 并且$\sum^{n}_{j} w_{i,j} == 1$. 值得一提的是, 这里的$w_{i, j}$ 并不是一个需要神经网络学习的参数，它是来源于 $x_i$ 和 $x_j$的之间的计算的结果。 它们之间最简单的一种计算方式，就是使用点积的方式
$$w'_{i,j} = {x_i}^{T} * x_j$$

这个点积的输出的取值范围在负无穷和正无穷之间，所以我们要使用一个softmax把它映射到[0, 1]之间, 并且要确保它们对于整个序列而言的和为1
$$w_{i, j} = softmax(w'_{i, j}) = \frac{e^{w'_{i,j}}}{\sum^n_{j} e^{w'_{i,j}}}$$

## Attention 和 self-attention 的区别是什么 ？
这里有几个重要的区别，可以帮助大家更好的区分在不同任务中的使用方法：

1. 在神经网络中，通常来说你会有输入层（input），应用激活函数后的输出层（output），在RNN当中你会有状态（state）。如果attention (AT) 被应用在某一层的话，它更多的是被应用在输出或者是状态层上，而当我们使用self-attention（SA），这种注意力的机制更多的实在关注input上。

2. Attention (AT) 经常被应用在从编码器（encoder）转换到解码器（decoder）。比如说，解码器的神经元会接受一些AT从编码层生成的输入信息。在这种情况下，AT连接的是两个不同的组件（component），编码器和解码器。但是如果我们用SA，它就不是关注的两个组件，它只是在关注你应用的那一个组件。那这里他就不会去关注解码器了，就比如说在Bert中，使用的情况，我们就没有解码器。

3. SA可以在一个模型当中被多次的、独立的使用（比如说在Transformer中，使用了18次；在Bert当中使用12次）。但是，AT在一个模型当中经常只是被使用一次，并且起到连接两个组件的作用。

4. SA比较擅长在一个序列当中，寻找不同部分之间的关系。比如说，在词法分析的过程中，能够帮助去理解不同词之间的关系。AT却更擅长寻找两个序列之间的关系，比如说在翻译任务当中，原始的文本和翻译后的文本。这里也要注意，在翻译任务重，SA也很擅长，比如说Transformer。

5. AT可以连接两种不同的模态，比如说图片和文字。SA更多的是被应用在同一种模态上，但是如果一定要使用SA来做的话，也可以将不同的模态组合成一个序列，再使用SA。

6. 对我来说，大部分情况，SA这种结构更加的general，在很多任务作为降维、特征表示、特征交叉等功能尝试着应用，很多时候效果都不错。

# Scaled-Dot Attention

\begin{equation}
Attention(Q, K, V) = softmax(\frac{QK^{T}}{\sqrt{d_k}}) * V
\end{equation}

其中 $Q \in \mathbb{R}^{n \times d_k}, K \in \mathbb{R}^{m \times d_k}, V \in \mathbb{R}^{m \times d_k}$. softmax 则是在 m 的那一维进行归一化。而自注意力，则是对于同一个 $X \in \mathbb{R}^{n \times d}$，通过不同的投影矩阵得到:



$$Q = X * W_q, \; \text{where} \; W_q \in \mathbb{R}^{d \times R'} $$

$$K = X * W_k, \; \text{where} \; W_k\in \mathbb{R}^{d \times R'} $$

$$V = X * W_v, \; \text{where} \; W_v \in \mathbb{R}^{d \times R'} $$


至于 Multi-Head Attention，则不过是 Attention 运算在不同的参数下重复多次然后将多个输出拼接起来，属于比较朴素的增强。




1. https://mp.weixin.qq.com/s?__biz=MzIwMTc4ODE0Mw==&mid=2247508324&idx=1&sn=0238864a68d94e7057574bee65315c6c&chksm=96ea7ee4a19df7f2332408475299041a876ff87ed0431bbb56736755ff3fc9fd5fbed29d6aa2&scene=21#wechat_redirect


2. https://mp.weixin.qq.com/s?__biz=MzIwMTc4ODE0Mw==&mid=2247486960&idx=1&sn=1b4b9d7ec7a9f40fa8a9df6b6f53bbfb&chksm=96e9d270a19e5b668875392da1d1aaa28ffd0af17d44f7ee81c2754c78cc35edf2e35be2c6a1&token=1416940766&lang=zh_CN&scene=21#wechat_redirect

3. https://mp.weixin.qq.com/s/zBUowlIU0JezWRnFBKYbug